Pre-processing algorithms 

In [1]:
import cv2 as cv
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

def save_image(name, img):
    cv.imwrite('..\datasets\ISIC2017\Training_data_processed\\successfully_cutted\\{}.jpg'.format(name), img)

def save_in_test(name, img):
    cv.imwrite('..\datasets\ISIC2017\Training_data_processed\\tests\\{}.jpg'.format(name), img)

def read_image(name):
    img = cv.imread('..\datasets\ISIC2017\Training_data_processed\\successfully_cutted\\{}.jpg'.format(name))
    return img


In [2]:
# Shades of Gray in a using color constancy algorithms that can correct or minimize color variation between images
# Goyal, Codela, Gessert e Pacheco

def shades_of_gray(img, power=6, gamma=None):
    img_dtype = img.dtype

    if gamma is not None:
        img = img.astype('uint8')
        look_up_table = np.ones((256,1), dtype='uint8') * 0
        for i in range(256):
            look_up_table[i][0] = 255 * pow(i/255, 1/gamma)
        img = cv.LUT(img, look_up_table)

    img = img.astype('float32')
    img_power = np.power(img, power)
    rgb_vec = np.power(np.mean(img_power, (0,1)), 1/power)
    rgb_norm = np.sqrt(np.sum(np.power(rgb_vec, 2.0)))
    rgb_vec = rgb_vec/rgb_norm
    rgb_vec = 1/(rgb_vec*np.sqrt(3))
    img = np.multiply(img, rgb_vec)

    img = np.clip(img, a_min=0, a_max=255)
    
    return img.astype(img_dtype)

In [3]:
# Gaussian filter to minimize the presence of artifacts
# It makes the image loses details, we have to evaluate if it is good to use it or not
# Khouloud

def gaussian_filter(img):
    return cv.GaussianBlur(img,(5,5),0)

In [4]:
def simple_contrast_enhancement(img):
    # converting to LAB color space
    lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    l_channel, a, b = cv.split(lab)

    # Applying CLAHE to L-channel
    # maybe add the limit and grid size as parameters:
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv.merge((cl,a,b))

    return cv.cvtColor(limg, cv.COLOR_LAB2BGR)


In [5]:
# Sarkar

# Takes a long time 
def non_local_mean_denoising(img):
    return cv.fastNlMeansDenoisingColored(img,None,10,10,7,21) # recommended values

In [6]:
#Afza
#adap_stack

def global_and_local_enhancement(img):
    return img

In [7]:
#Khan

def abc_contrast_enhancement(img):
    return img

In [8]:
img_names = ['ISIC_0000001', 'ISIC_0000002', 'ISIC_0000003', 'ISIC_0000004']

for img_name in img_names:
    img = read_image(img_name + '_cutted')
    save_in_test(img_name, img)
    img = non_local_mean_denoising(img)
    save_in_test(img_name + '_sog', img)

[ WARN:0@1.461] global loadsave.cpp:244 findDecoder imread_('..\datasets\ISIC2017\Training_data_processed\successfully_cutted\ISIC_0000001_cutted.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
